## ЦИФРОВОЙ ПРОРЫВ 2022
# **Чемпионат в республике Саха (Якутия)**
# Разработка алгоритма по прогнозированию карьерной траектории сотрудника
Технологии искусственного интеллекта трансформируют сферу управления персоналом, позволяя решать практические HR-задачи. Уже сейчас умные алгоритмы могут подобрать кадры, спланировать размер фонда заработной платы, выбрать образовательные курсы, оценить мотивацию, эффективность труда и даже спрогнозировать карьерное развитие специалиста.

Аналитика по определению карьерной траектории строится, в том числе, на основе данных по использованию различных корпоративных информационных систем (система мониторинга рабочего времени, сервисы видеоконференции связи, IP-телефонии, СЭД, СКУД, электронная почта). Таким образом, для некоторых профессий, связанных с активным использованием электронных устройств (программист, бухгалтер, маркетолог и т.д.), можно выделить цифровой профиль успешных специалистов, выполняющих задачи своевременно и качественно и получающих новые должности.

На основе совокупности информации о кадровых назначениях, активности и вовлеченности сотрудников в производственный процесс, участникам чемпионата нужно предложить алгоритм по прогнозу изменений должности работников.

# **Data analysis**
**Импортируем все необходимые библиотеки и модули**

In [ ]:
import pandas as pd
import numpy as np
import time

from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

**Загрузка данных в датафреймы**

In [ ]:
df_train = pd.read_csv("../input/yakutsk/train_dataset_train.csv")
df_test = pd.read_csv("../input/yakutsk/test_dataset_test.csv")
calls = pd.read_csv("../input/yakutsk/Calls.csv")
connect = pd.read_csv("../input/yakutsk/ConnectionTime.csv")
educat = pd.read_csv("../input/yakutsk/Education.csv")
skud = pd.read_csv("../input/yakutsk/SKUD.csv")
tasks = pd.read_csv("../input/yakutsk/Tasks.csv")
timen = pd.read_csv("../input/yakutsk/TimenNetwork.csv")
working = pd.read_csv("../input/yakutsk/WorkingDay.csv")

**Смотрим первые записи**

In [2]:
df_train.head(3)

NameError: name 'df_train' is not defined

# **Подготовка данных**

**Записываем id в отдельный лист**

In [ ]:
test_id = df_test[["id"]]

**Считаем среднее количество звонков в день и среднее количество времени за телефоном**

In [ ]:
calls["CallTime"] = calls["CallTime"].apply(lambda x: float(x.replace(',', '.')))

In [ ]:
calls["avg_calls"] = calls.groupby(["id"])["NumberOfCalls"].transform('mean')
calls["avg_calltime"] = calls.groupby(["id"])["CallTime"].transform('mean')

In [ ]:
calls.head(3)

# **Работа с таблицей connect**
Параметр *late_count* будет хранить общее количество времени опоздания

In [3]:
connect["Время опоздания"] = connect["Время опоздания"].apply(lambda x: float(str(x).replace(',', '.')))
connect["Время опоздания"] = connect["Время опоздания"].apply(lambda x: float(str(x).replace('nan', '0')))

NameError: name 'connect' is not defined

In [ ]:
connect["late_count"] = connect.groupby(["id"])["Время опоздания"].transform('sum')

In [ ]:
connect.head(3)

# **Работа с tasks**
Категоризируем параметр "статус по просрочке", чтобы 0 обозначал, что задание выполнено в срок, а 1 - просроченное. Параметр *overdue_count* будет обозначать количество просроченнных заданий.

In [4]:
tasks["Статус по просрочке"] = tasks["Статус по просрочке"].astype("category").cat.codes
tasks["overdue_count"] = tasks.groupby(["id"])["Статус по просрочке"].transform('sum')

NameError: name 'tasks' is not defined

In [ ]:
tasks.head(3)

# **Работа с working**
Создаём новый параметр *monitorTime_count* - общее время активности

In [ ]:
working["monitorTime_count"] = working.groupby(["id"])["monitorTime"].transform('sum')

In [ ]:
working.head(3)

# **Работа с timen**
Создаём новый параметр *monitorTime_avg* - среднее время активности

In [5]:
timen["monitorTime_avg"] = timen.groupby(["id"])["monitor_Time"].transform('mean')

NameError: name 'timen' is not defined

In [ ]:
timen.head(3)

# **Работа с skud**
Категоризируем столбец "Вых/Будни" и считаем количество проработанных дней в новый параметр *days_count*

In [6]:
skud["Вых/Будни"] = skud["Вых/Будни"].astype("category").cat.codes
skud["days_count"] = skud.groupby(["id"])["Вых/Будни"].transform('count')

NameError: name 'skud' is not defined

In [ ]:
skud.head(3)

# **Работа с educat**
Пронумеруем каждый вид образования и создадим несколько новых параметров:
* retraining - заменяем на ноль, если образование не является переподготовкой, иначе оставляем присвоенный номер
* Вид образования - меняем на ноль, если образование это переподготовка, иначе оставляем присвоенный номер
* educat_count - Количество образований
* chief - 1, если человек является руководителем
* kind_educat - "лучшее" образование
* retraining_count - количество переподготовок
* number_chief - категоризация руководителей

In [7]:
educat["Вид образования"] = educat["Вид образования"].fillna(-1)
educat["Вид образования"] = educat["Вид образования"].map({"Высшее образование - бакалавриат": 5,
                                                                      "Среднее профессиональное образование": 4,
                                                                      "Высшее образование": 5,
                                                                      "Повышение квалификации": 8,
                                                                      "Переподготовка": 7, "Послевузовское образование": 7,
                                                                      "Дополнительное профессиональное образование": 7,
                                                                      "Высшее образование - специалитет, магистратура": 5,
                                                                      "Среднее общее образование": 2,
                                                                      "Начальное профессиональное образование": 2,
                                                                      "Аспирантура": 6,
                                                                      "Среднее (полное) общее образование": 2,
                                                                      "Неполное высшее образование": 3,
                                                                      "Основное общее образование": 2,
                                                                      "Профессиональное обучение": 3,
                                                                      "Начальное общее образование": 1})
educat = educat.drop_duplicates()
educat["retraining"] = educat["Вид образования"].apply(lambda x: x if x > 6 else 0)
educat["Вид образования"] = educat["Вид образования"].apply(lambda x: x if x <= 6 else 0)
educat["educat_count"] = educat["id"].map(educat[educat["Вид образования"] != 0].groupby(["id"])["Вид образования"].count())

NameError: name 'educat' is not defined

In [ ]:
educat["chief"] = educat["Табельный номер руководителя"] == educat["id"]
educat["chief"] = educat["chief"].astype("int")
educat["kind_educat"] = educat.groupby("id")["Вид образования"].transform("max")
educat["retraining_count"] = educat.groupby("id")["retraining"].transform("count")
educat["Табельный номер руководителя"].fillna(-1)
educat["number_chief"] = educat["Табельный номер руководителя"].astype("category").cat.codes
educat = educat[["id", "retraining_count", "educat_count", "chief", "number_chief"]]
educat = educat.drop_duplicates()

In [ ]:
educat.head(3)

# **Подготовка датасетов для тренировки**

**Сливаем нужные столбцы в один датасет**

In [8]:
train1 = df_train.merge(calls[["id", "avg_calls","avg_calltime"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test1 = test_id.merge(calls[["id", "avg_calls","avg_calltime"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

NameError: name 'df_train' is not defined

In [ ]:
train2 = train1.merge(connect[["id", "late_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test2 = test1.merge(connect[["id", "late_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

In [ ]:
train3 = train2.merge(tasks[["id", "overdue_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test3 = test2.merge(tasks[["id", "overdue_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

In [ ]:
train4 = train3.merge(working[["id", "monitorTime_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test4 = test3.merge(working[["id", "monitorTime_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

In [9]:
train5 = train4.merge(timen[["id", "monitorTime_avg"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test5 = test4.merge(timen[["id", "monitorTime_avg"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

NameError: name 'train4' is not defined

In [ ]:
train6 = train5.merge(skud[["id", "days_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test6 = test5.merge(skud[["id", "days_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

In [ ]:
train7 = train6.merge(educat, how="left", on="id").drop_duplicates().reset_index(drop=True)
test7 = test6.merge(educat, how="left", on="id").drop_duplicates().reset_index(drop=True)

In [ ]:
trainFinal = train7.fillna(0)
testFinal = test7.fillna(0)
trainFinal.head(3)

In [10]:
testFinal.head(3)

NameError: name 'testFinal' is not defined

In [ ]:
X = trainFinal.drop(["id", "type"], axis=1).values
Y = trainFinal["type"].values

In [ ]:
X_test = testFinal.drop(["id"], axis=1).values

# **Over-sampling (устранение дисбаланса классов)**
Для устранение дисбаланса классов воспользуемся SMOTE из библиотеки imblearn, этот алгоритм создает дополнительные синтетические наблюдения меньших классов.

In [11]:
oversample = SMOTE()
X, Y = oversample.fit_resample(X, Y)

NameError: name 'SMOTE' is not defined

# **Machine Learning**
В качестве модели были использованы алгоритмы RandomForestClassifier, Support Vector Classification и Logistic Regression. Затем прибегли к использованию ансамбля моделей, когда результаты сразу нескольких из них участвуют в формировании конечного результата.

In [ ]:
ss = StandardScaler()
X_scaled = ss.fit_transform(X)
X_t_scaled = ss.transform(X_test)

In [ ]:
clf1 = RandomForestClassifier(n_estimators=100)
clf2 = make_pipeline(ss, SVC(gamma='auto', probability=True))
clf3 = LogisticRegression(random_state=0)
eclf = VotingClassifier(estimators=[('rfc', clf1), ('svc', clf2), ('logreg', clf3)], voting='soft')
eclf = eclf.fit(X_scaled, Y)
Y_pred = eclf.predict(X_t_scaled)

# **Формировка результатов**

In [12]:
sub = pd.DataFrame({'id':test_id["id"], 'type':Y_pred})
sub.to_csv('VotingClassifier (rfc, svc, logreg) v2.csv', index = False)

NameError: name 'pd' is not defined